Jeśli chcesz mieć dużo i legalnie, to zamiast gazet z paywallem warto dodać:
Bankier.pl – darmowe, często cytują ekonomistów („NBP przewiduje inflację 3,8% w 2025 r.”).
 Teksty łatwe do scrapowania.


Money.pl – często piszą: „Według analityków X prognoza wzrostu PKB wyniesie 2,7%...”.


Business Insider Polska – ma wiele otwartych artykułów, szczególnie sekcja „Gospodarka”.


Forsal.pl / Dziennik Gazeta Prawna – część tekstów otwarta, można brać akapity.


Interia Biznes, Wirtualna Polska Gospodarka – popularne serwisy z cytatami ekonomistów.


PAP Biznes (biznes.pap.pl) – krótkie notki agencyjne z konkretnymi liczbami, idealne do ekstrakcji.


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
from urllib.parse import urlparse
import time
import re
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; MagisterkaBot/0.1; +https://example.com/contact)"
}


def fetch_html(url: str, sleep: float = 1.0) -> str:
    """Pobiera HTML z podanego URL z prostym backoffem."""
    resp = requests.get(url, headers=HEADERS, timeout=15)
    resp.raise_for_status()
    time.sleep(sleep)
    return resp.text


def parse_date(raw: str) -> date | None:
    """Próbuje sparsować datę z różnych formatów spotykanych w PL mediach."""
    if not raw:
        return None
    raw = raw.strip()

    fmts = [
        "%Y-%m-%d",
        "%Y-%m-%d %H:%M",
        "%Y-%m-%d %H:%M:%S",
        "%d.%m.%Y",
        "%d.%m.%Y %H:%M",
        "%d-%m-%Y",
        "%d %m %Y",
        "%d %b %Y",
        "%d %B %Y",
    ]
    for fmt in fmts:
        try:
            return datetime.strptime(raw, fmt).date()
        except ValueError:
            continue

    m = re.search(r"\b(\d{4})-(\d{2})-(\d{2})\b", raw)
    if m:
        y, mth, d = map(int, m.groups())
        return date(y, mth, d)

    m = re.search(r"\b(\d{1,2})[./-](\d{1,2})[./-](\d{4})\b", raw)
    if m:
        d, mth, y = map(int, m.groups())
        try:
            return date(y, mth, d)
        except ValueError:
            return None

    return None


def search_date_in_text(text: str) -> date | None:
    """Szukamy pierwszego wystąpienia daty w tekście."""
    m = re.search(r"\b(20[12]\d)-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])\b", text)
    if m:
        y, mth, d = map(int, m.groups())
        return date(y, mth, d)

    m = re.search(r"\b(0?[1-9]|[12]\ ")


In [11]:
import re
from urllib.parse import urlparse
from datetime import datetime
import requests
from bs4 import BeautifulSoup

SESSION = requests.Session()
SESSION.headers.update({
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0 Safari/537.36"
})

DATE_PATTERNS = [
    re.compile(r"\b\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}(?::\d{2})?)?\b"),  # 2023-09-01 12:34
    re.compile(r"\b\d{1,2}[./-]\d{1,2}[./-]\d{4}(?:[ ,T]+\d{2}:\d{2}(?::\d{2})?)?\b")  # 01.09.2023, 12:34
]

def _is_bankier(url: str) -> bool:
    host = urlparse(url).netloc.lower()
    return "bankier.pl" in host

def _find_date_in_text(text: str) -> str | None:
    if not text:
        return None
    t = " ".join(text.split())
    for rx in DATE_PATTERNS:
        m = rx.search(t)
        if m:
            return m.group(0)
    return None

def _extract_bankier_date(soup: BeautifulSoup) -> str | None:
    # Search in class a-span
    for el in soup.select("span.a-span"):
        txt = el.get_text(strip=True)
        d = _find_date_in_text(txt) or txt
        if d:
            # Return the first valid date
            return d
    return None

def _extract_meta_date(soup: BeautifulSoup) -> str | None:
    # Simple fallback with meta/time
    meta_candidates = [
        ("meta", {"property": "article:published_time"}, "content"),
        ("meta", {"name": "article:published_time"}, "content"),
        ("meta", {"itemprop": "datePublished"}, "content"),
        ("meta", {"name": "date"}, "content"),
    ]
    for tag, attrs, attr_name in meta_candidates:
        el = soup.find(tag, attrs=attrs)
        if el and el.has_attr(attr_name):
            d = _find_date_in_text(el.get(attr_name, "")) or el.get(attr_name, "")
            if d:
                return d
    time_el = soup.find("time")
    if time_el:
        if time_el.has_attr("datetime"):
            d = _find_date_in_text(time_el["datetime"]) or time_el["datetime"]
            if d:
                return d
        txt = time_el.get_text(strip=True)
        d = _find_date_in_text(txt) or txt
        if d:
            return d
    return None

def _extract_title(soup: BeautifulSoup) -> str | None:
    og = soup.find("meta", property="og:title")
    if og and og.get("content"):
        return og["content"].strip()
    h1 = soup.find("h1")
    if h1:
        return h1.get_text(strip=True)
    if soup.title and soup.title.string:
        return soup.title.string.strip()
    return None

def _extract_text(soup: BeautifulSoup) -> str:
    # Collect paragraphs from <article>, then globally
    parts = []
    article = soup.find("article")
    if article:
        parts = [p.get_text(" ", strip=True) for p in article.find_all("p")]
    if not parts:
        parts = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
    return "\n\n".join([t for t in parts if t])

def extract_generic_article(url: str) -> dict:
    resp = SESSION.get(url, timeout=20)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    title = _extract_title(soup)
    if _is_bankier(url):
        date = _extract_bankier_date(soup)
    else:
        date = _extract_meta_date(soup)

    text = _extract_text(soup)
    return {"title": title, "date": date, "text": text}

In [13]:
if __name__ == "__main__":
    url = "https://www.bankier.pl/wiadomosc/Kuczynski-Recesja-w-2023-roku-to-pewniak-ekonomistow-Polska-moze-miec-jednak-szczescie-8463778.html"
    
    article = extract_generic_article(url)
    
    print("\n===== TYTUŁ =====")
    print(article["title"])
    
    print("\n===== DATA =====")
    print(article["date"])
    
    print("\n===== FRAGMENT TEKSTU =====")
    print(article["text"][:1000])  # pokaż tylko pierwsze 1000 znaków, żeby nie zalało konsoli



===== TYTUŁ =====
Kuczyński: Recesja w 2023 roku to "pewniak" ekonomistów. Polska może mieć jednak szczęście

===== DATA =====
2022-12-31 06:35

===== FRAGMENT TEKSTU =====
Polska gospodarka trzyma się zaskakująco dobrze. Dane za listopad okazały się w większości lepsze od prognozowanych przez ekonomistów, mimo wysokiej na tle innych krajów inflacji, i większość przewidywań na 2023 rok nie zakłada spadku PKB. Średnio mowa jest o wzroście o 0,5 proc., choć są i wskazania nawet trzykrotnie bardziej optymistyczne. Wiele zależeć będzie od kursu eurodolara i sytuacji w gospodarce niemieckiej, która z kolei nakierowana jest na eksport do Chin.

− Scenariusze na 2023 rok są właściwie dwa: albo recesja, ale wtedy płytka i stosunkowo krótka, np. 1-2 kwartały, albo brak recesji i bardzo wolny wzrost gospodarczy. Rząd zakłada, że będzie to wolny wzrost gospodarczy - o 1,7 proc. PKB . To jest bardzo maleńko, ale wiele innych instytucji finansowych mówi o niewielkim spadku, więc fifty-fifty - mówi